In [3]:
#import dependencies
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey

In [4]:
#create classes that will be our tables for the measurements and stations
Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'
    station = Column(String(20), primary_key=True)
    station_name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

#need to see how to map a foreign key here in the classes and map it to Station.station
    
class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(20), ForeignKey('stations.station'))
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

In [5]:
#create the database
engine = create_engine('sqlite:///hawaii.sqlite', echo=True)

In [6]:
#send our table metadata to the newly created databse to create the tables
#from our two classes
Base.metadata.create_all(bind=engine)

2018-05-03 21:06:47,203 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-05-03 21:06:47,214 INFO sqlalchemy.engine.base.Engine ()
2018-05-03 21:06:47,217 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-05-03 21:06:47,218 INFO sqlalchemy.engine.base.Engine ()
2018-05-03 21:06:47,226 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stations")
2018-05-03 21:06:47,229 INFO sqlalchemy.engine.base.Engine ()
2018-05-03 21:06:47,233 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurements")
2018-05-03 21:06:47,234 INFO sqlalchemy.engine.base.Engine ()
2018-05-03 21:06:47,240 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE stations (
	station VARCHAR(20) NOT NULL, 
	station_name VARCHAR(255), 
	latitude FLOAT, 
	longitude FLOAT, 
	elevation FLOAT, 
	PRIMARY KEY (station)
)


2018-05-03 21:06:47,242 INFO sqlalchemy.engine.base.Engine ()
2018-05-03 21:06:47,287 INFO sqlalchemy.en

In [7]:
#import the dataframes that will be mapped to the tables in the database
stations_df = pd.read_csv('Resources\clean_hawaii_stations.csv')

In [8]:
#renaming the column 'name' to 'stationname' because for some reasion when iterating over 'name'
#it was returning only an index value
stations_df['stationname'] = stations_df['name']

In [9]:
stations_df

,Unnamed: 0,station,name,latitude,longitude,elevation,stationname
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0,"WAIKIKI 717.2, HI US"
1,1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6,"KANEOHE 838.1, HI US"
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0,"KUALOA RANCH HEADQUARTERS 886.9, HI US"
3,3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9,"PEARL CITY, HI US"
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6,"UPPER WAHIAWA 874.3, HI US"
5,5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5,"WAIMANALO EXPERIMENTAL FARM, HI US"
6,6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9,"WAIHEE 837.5, HI US"
7,7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9,"HONOLULU OBSERVATORY 702.2, HI US"
8,8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4,"MANOA LYON ARBO 785.2, HI US"


In [10]:
from sqlalchemy.orm import Session
session = Session(engine)

In [11]:
#iterate over each dataframe row and add this to the database tables
for index, row in stations_df.iterrows():
    station = Station(station=row.station, station_name=row.stationname, latitude=row.latitude,\
            longitude=row.longitude, elevation=row.elevation)
    session.add(station)
session.commit()

2018-05-03 21:06:47,611 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-05-03 21:06:47,613 INFO sqlalchemy.engine.base.Engine INSERT INTO stations (station, station_name, latitude, longitude, elevation) VALUES (?, ?, ?, ?, ?)
2018-05-03 21:06:47,615 INFO sqlalchemy.engine.base.Engine (('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6), ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0), ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9), ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6), ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5), ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9), ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9), ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4))
2018-05-03 21:06:47,626 INFO sqlalc

In [12]:
engine.execute('select * from stations').fetchall()

2018-05-03 21:06:47,708 INFO sqlalchemy.engine.base.Engine select * from stations
2018-05-03 21:06:47,712 INFO sqlalchemy.engine.base.Engine ()


[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

Comment: our database looks good - looks like it was populated correctly

In [13]:
#now read in the measurements dataframe and send all its data to the measurements table
meas_df = pd.read_csv('Resources\clean_hawaii_measurements.csv')

In [21]:
meas_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.00,73


In [15]:
for index, row in meas_df.iterrows():
    measurement = Measurement(station=row.station, date=row.date, prcp=row.prcp,\
            tobs=row.tobs)
    session.add(measurement)
session.commit()

2018-05-03 21:06:54,685 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-05-03 21:06:54,842 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:54,843 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-01-01', 0.08, 65)
2018-05-03 21:06:54,854 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:54,855 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-01-02', 0.0, 63)
2018-05-03 21:06:54,857 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:54,858 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-01-03', 0.0, 74)
2018-05-03 21:06:54,859 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:54,865 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-01-04', 0.0, 76)
201

2018-05-03 21:06:55,018 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-02-09', 0.0, 73)
2018-05-03 21:06:55,020 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,021 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-02-11', 0.0, 73)
2018-05-03 21:06:55,137 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,139 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-02-12', 0.02, 69)
2018-05-03 21:06:55,140 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,142 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-02-13', 0.01, 69)
2018-05-03 21:06:55,146 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,149 INFO sqlalchemy.engine.base.Engine ('USC00519397', '20

2018-05-03 21:06:55,293 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-03-24', 0.0, 74)
2018-05-03 21:06:55,306 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,314 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-03-26', 0.0, 72)
2018-05-03 21:06:55,315 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,316 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-03-27', 0.0, 72)
2018-05-03 21:06:55,318 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,319 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-03-28', 0.0, 73)
2018-05-03 21:06:55,323 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,324 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010

2018-05-03 21:06:55,487 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-05-04', 0.08, 76)
2018-05-03 21:06:55,491 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,492 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-05-05', 0.01, 76)
2018-05-03 21:06:55,495 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,497 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-05-06', 0.0, 80)
2018-05-03 21:06:55,501 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,502 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-05-07', 0.05, 74)
2018-05-03 21:06:55,504 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,505 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2

2018-05-03 21:06:55,649 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-06-11', 0.0, 78)
2018-05-03 21:06:55,652 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,655 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-06-12', 0.0, 78)
2018-05-03 21:06:55,656 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,658 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-06-13', 0.0, 78)
2018-05-03 21:06:55,660 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,662 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010-06-14', 0.0, 77)
2018-05-03 21:06:55,665 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:06:55,668 INFO sqlalchemy.engine.base.Engine ('USC00519397', '2010

...

2018-05-03 21:08:33,552 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-03-19', 0.0, 74)
2018-05-03 21:08:33,553 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,554 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-03-20', 0.0, 74)
2018-05-03 21:08:33,555 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,556 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-03-21', 0.0, 72)
2018-05-03 21:08:33,557 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,558 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-03-22', 0.0, 72)
2018-05-03 21:08:33,560 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,561 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017

2018-05-03 21:08:33,653 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-04-26', 0.22, 77)
2018-05-03 21:08:33,655 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,656 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-04-27', 0.11, 76)
2018-05-03 21:08:33,657 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,661 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-04-28', 0.79, 71)
2018-05-03 21:08:33,663 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,664 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-04-29', 0.0, 68)
2018-05-03 21:08:33,665 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,666 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2

2018-05-03 21:08:33,851 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-06-06', 0.0, 74)
2018-05-03 21:08:33,852 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,853 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-06-07', 0.01, 79)
2018-05-03 21:08:33,855 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,856 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-06-08', 0.0, 75)
2018-05-03 21:08:33,857 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,858 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-06-10', 0.0, 72)
2018-05-03 21:08:33,860 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,861 INFO sqlalchemy.engine.base.Engine ('USC00516128', '201

2018-05-03 21:08:33,954 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-07-13', 0.68, 77)
2018-05-03 21:08:33,956 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,957 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-07-14', 0.68, 76)
2018-05-03 21:08:33,958 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,959 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-07-15', 0.1, 78)
2018-05-03 21:08:33,961 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,962 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-07-16', 0.5, 75)
2018-05-03 21:08:33,964 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:33,965 INFO sqlalchemy.engine.base.Engine ('USC00516128', '20

2018-05-03 21:08:34,063 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-08-19', 0.09, 71)
2018-05-03 21:08:34,064 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:34,066 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-08-20', 0.0, 78)
2018-05-03 21:08:34,067 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:34,068 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-08-21', 0.56, 76)
2018-05-03 21:08:34,069 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:34,070 INFO sqlalchemy.engine.base.Engine ('USC00516128', '2017-08-22', 0.5, 76)
2018-05-03 21:08:34,071 INFO sqlalchemy.engine.base.Engine INSERT INTO measurements (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-05-03 21:08:34,072 INFO sqlalchemy.engine.base.Engine ('USC00516128', '20

In [19]:
#check one row to ensure we have data
engine.execute('select * from measurements').first()

2018-05-03 21:16:17,340 INFO sqlalchemy.engine.base.Engine select * from measurements
2018-05-03 21:16:17,341 INFO sqlalchemy.engine.base.Engine ()


(1, 'USC00519397', '2010-01-01', 0.08, 65)